<a href="https://colab.research.google.com/github/020/020.github.io/blob/master/23_12_22_Presentations_LSTM_Exp_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
import warnings
warnings.filterwarnings("ignore")

import plotly.offline as pyoff
import plotly.graph_objs as go


In [36]:
def parser(x):
    return pd.datetime.strptime(x,'%d/%m/%Y')

df = pd.read_csv('/content/ed_presentations_from_aug2020.csv', parse_dates=[0], index_col=0, date_parser=parser)
df.tail()


,y
ds,
2022-11-12,2967
2022-11-13,2960
2022-11-14,3375
2022-11-15,3085
2022-11-16,3183


In [37]:
train = df

scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)

In [38]:

n_input = 12
n_features = 1
generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)

In [39]:
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')

In [40]:
history = model.fit_generator(generator,epochs=100,verbose=1)

Epoch 1/100
138/138 [==============================] - 4s 18ms/step - loss: 0.0204
Epoch 2/100
138/138 [==============================] - 2s 18ms/step - loss: 0.0147
Epoch 3/100
138/138 [==============================] - 3s 18ms/step - loss: 0.0140
Epoch 4/100
138/138 [==============================] - 3s 18ms/step - loss: 0.0144
Epoch 5/100
138/138 [==============================] - 3s 19ms/step - loss: 0.0136
Epoch 6/100
138/138 [==============================] - 3s 18ms/step - loss: 0.0142
Epoch 7/100
138/138 [==============================] - 3s 18ms/step - loss: 0.0130
Epoch 8/100
138/138 [==============================] - 3s 18ms/step - loss: 0.0125
Epoch 9/100
138/138 [==============================] - 2s 18ms/step - loss: 0.0125
Epoch 10/100
138/138 [==============================] - 2s 18ms/step - loss: 0.0123
Epoch 11/100
138/138 [==============================] - 3s 18ms/step - loss: 0.0124
Epoch 12/100
138/138 [==============================] - 3s 18ms/step - loss: 0.0120
E

In [41]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

plot_data = [
    go.Scatter(
        x=hist['epoch'],
        y=hist['loss'],
        name='loss'
    )
    
]

plot_layout = go.Layout(
        title='Training loss'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [42]:
pred_list = []

batch = train[-n_input:].reshape((1, n_input, n_features))

for i in range(n_input):   
    pred_list.append(model.predict(batch)[0]) 
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

1/1 [==============================] - 0s 18ms/step


In [47]:
add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,13) ]
future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)

In [48]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                          index=future_dates[-n_input:].index, columns=['Prediction'])

df_proj = pd.concat([df,df_predict], axis=1)

df_proj.tail(12)

,y,Prediction
2022-11-17,NaN,3137.184633
2022-11-18,NaN,3112.651993
2022-11-19,NaN,3051.855205
2022-11-20,NaN,3108.689118
2022-11-21,NaN,3368.413718
2022-11-22,NaN,3238.509712
2022-11-23,NaN,3160.030918
2022-11-24,NaN,3190.881098
2022-11-25,NaN,3173.833324
2022-11-26,NaN,3138.602333


In [49]:
plot_data = [
    go.Scatter(
        x=df_proj.index,
        y=df_proj['y'],
        name='actual'
    ),
    go.Scatter(
        x=df_proj.index,
        y=df_proj['Prediction'],
        name='prediction'
    )
]

plot_layout = go.Layout(
        title='Presentation prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)